In [1]:
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
import pandas as pd
import urllib.request
from shapely.geometry import Point, shape
import json
import math
import networkx as nx

In [2]:
def get_haversine_distance(point_1, point_2):
    """
    Calculate the distance between any 2 points on earth given as [lon, lat]
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [point_1[0], point_1[1], 
                                                point_2[0], point_2[1]])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [73]:
city='Detroit'
SIM_AREA_PATH='./cities/'+city+'/clean/table_area.geojson'
FLOYD_PREDECESSOR_PATH='./cities/'+city+'/clean/fw_result.json'
INT_NET_DF_FLOYD_PATH='./cities/'+city+'/clean/sim_net_df_floyd.csv'
INT_NET_COORDINATES_PATH='./cities/'+city+'/clean/sim_net_node_coords.json'

sim_area=json.load(open(SIM_AREA_PATH))

full_area=[shape(f['geometry']) for f in sim_area['features']]
bounds=[shp.bounds for shp in full_area]
boundsAll=[min([b[0] for b in bounds]), #W
               min([b[1] for b in bounds]), #S
               max([b[2] for b in bounds]), #E
               max([b[3] for b in bounds])] #N

In [4]:
query='http://api.openstreetmap.org/api/0.6/map?bbox='+','.join([str(b) for b in boundsAll])
file = urllib.request.urlopen(query)
xData=et.parse(file)
file.close()

root=xData.getroot()
root.tag

nodeLat=[]
nodeLong=[]
nodeId=[]
aNodes=[]
bNodes=[]
hwTags=[]
names=[]
oneway=[]
ref=[]
maxspeed=[]
osmid=[]

for child in root:
    if child.tag=='node':
        nodeLat.extend([child.get('lat')])
        nodeLong.extend([child.get('lon')])
        nodeId.extend([child.get('id')])
    elif child.tag=='way':
        highwayTag, name, oneW, re, mSp=[], [], [], [], []
        for t in child.findall('tag'):
            key=t.get('k')
            if key=='highway':
                value=t.get('v')
                highwayTag.extend([value])
        #now confirmed if the child is a way: if so, loop through the tags again and save them
        if any(highwayTag):
            for t in child.findall('tag'):
                if t.get('k')=='name':
                    name=t.get('v')
                if t.get('k')=='oneway':
                    oneW=t.get('v')
                if t.get('k')=='ref':
                    re=t.get('v')
                if t.get('k')=='maxspeed':
                    mSp=t.get('v')
            first=1
            for gChild in child:
                if gChild.tag=='nd':
                    if first==1:
                        #aNodes.extend([gChild.get('ref')])
                        lastNode=gChild.get('ref')
                    else:
                        aNodes.extend([lastNode])
                        lastNode=gChild.get('ref')
                        bNodes.extend([lastNode])
                        hwTags.extend([highwayTag[0]])
                        names.extend([name])
                        oneway.extend([oneW])
                        ref.extend([re])
                        maxspeed.extend([mSp])
                        osmid.extend([list(child.getiterator())[0].attrib['id']])
                    first=0

In [5]:
latLong= [[nodeLat[i], nodeLong[i]] for i in range(len(nodeId))]
LatLongDict = dict(zip(nodeId, latLong))

aNodeLat=[LatLongDict[aNodes[i]][0] for i in range(len(aNodes))]
aNodeLon=[LatLongDict[aNodes[i]][1] for i in range(len(aNodes))]
bNodeLat=[LatLongDict[bNodes[i]][0] for i in range(len(bNodes))]
bNodeLon=[LatLongDict[bNodes[i]][1] for i in range(len(bNodes))]

network=pd.DataFrame({'aNodes':aNodes, 'bNodes':bNodes, 
'aNodeLat':aNodeLat, 'aNodeLon':aNodeLon,'bNodeLat':bNodeLat,'bNodeLon':bNodeLon, 'type':hwTags, 
'name':names, 'speed':maxspeed, 'ref':ref, 'osmid':osmid, 'oneway':oneway})

Add link distances

In [6]:
network['distance']=network.apply(lambda row: get_haversine_distance([float(row['aNodeLon']), float(row['aNodeLat'])],
                                                                     [float(row['bNodeLon']), float(row['bNodeLat'])]), axis=1)

Add other-way nodes

In [7]:
copy_edges=network.copy()
for i in range(len(copy_edges)):
    if copy_edges.iloc[i]['oneway']!='yes':
        temp_link=copy_edges.iloc[i].copy()
        temp_from_node = temp_link['aNodes']
        temp_link['aNodes']=temp_link['bNodes']
        temp_link['bNodes']=temp_from_node
        network.loc[len(network)]=temp_link.copy()

Index by to and from nodes and remove duplicates

In [8]:
network['aNode_bNode']=network.apply(lambda row: '{}_{}'. format(row['aNodes'], row['bNodes']), axis=1)
network=network.set_index('aNode_bNode')
len(network)

12465

In [9]:
network = network.loc[~network.index.duplicated(keep='first')]
len(network)

12465

In [10]:
%matplotlib inline

In [11]:
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, Circle, Marker

In [12]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [13]:
network.head()

,aNodes,bNodes,aNodeLat,aNodeLon,bNodeLat,bNodeLon,type,name,speed,ref,osmid,oneway,distance
aNode_bNode,,,,,,,,,,,,,
62547437_62547439,62547437,62547439,42.3326578,-83.0620911,42.3330202,-83.0622765,motorway_link,[],[],[],8727497,yes,43.082564
62547439_62547441,62547439,62547441,42.3330202,-83.0622765,42.3332980,-83.0624543,motorway_link,[],[],[],8727497,yes,34.172964
62547441_62547447,62547441,62547447,42.3332980,-83.0624543,42.3334178,-83.0625246,motorway_link,[],[],[],8727497,yes,14.520527
62574668_62574679,62574668,62574679,42.3332176,-83.0727618,42.3333486,-83.0721618,motorway_link,[],[],[],8727643,yes,51.425975
62574679_62574682,62574679,62574682,42.3333486,-83.0721618,42.3333736,-83.0720108,motorway_link,[],[],[],8727643,yes,12.719632


# Find subset of links where at least one node is in the simulation area

In [14]:
print(len(LatLongDict))
sim_area_nodes=set()
# check if each node in any sim area, if so add to list
for node_num in LatLongDict:
    for feat in sim_area['features']:
        if shape(feat['geometry']).contains(Point(
            float(LatLongDict[node_num][1]), 
            float(LatLongDict[node_num][0]))):
            sim_area_nodes.add(node_num)
len(sim_area_nodes)

13745


8725

In [15]:
print(len(network))
network=network.loc[((network['aNodes'].isin(sim_area_nodes))|
    (network['bNodes'].isin(sim_area_nodes)))]
print(len(network))

12465
7994


# Plot network inside sim area only

In [16]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =15, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [17]:
edges_types=set(network['type'])
edges_types

{'footway',
 'pedestrian',
 'primary',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'tertiary',
 'track',
 'unclassified'}

In [18]:
drive_types=[t for t in edges_types if t not in ['footway', 'path', 'steps', 'pedestrian', 'track']]

In [19]:
drive_edges=network.loc[network['type'].isin(drive_types)]

In [20]:
len(drive_edges)

3604

In [21]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =15, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in drive_edges.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Create Networkx DiGraph

In [22]:
G=nx.DiGraph()
for i, row in drive_edges.iterrows():
    G.add_edge(row['aNodes'], row['bNodes'], 
               weight=row['distance'])

Solve shortest paths using Floyd Warshall

In [24]:
fw_result=nx.floyd_warshall_predecessor_and_distance(G, weight='weight')

In [52]:
# Create mapping from nodes to link attributes to speed up queries
nodes_to_link_attributes={}
for ind, row in drive_edges.iterrows():
    nodes_to_link_attributes['{}_{}'.format(row['aNodes'], row['bNodes'])]={
        'distance': row['distance'],
        'from_coord': [float(row['aNodeLon']), float(row['aNodeLat'])],
        'to_coord': [float(row['bNodeLon']), float(row['bNodeLat'])]}

In [53]:
def get_node_path_from_fw(all_pred, from_node, to_node):
    if from_node==to_node:
        return []
    pred=to_node
    path=[pred]
    while not pred==from_node:
        pred=all_pred[from_node][pred]
        path.insert(0,pred)
    return path
        
def get_path_coords_distances(drive_edges, path):
    coords, distances =[], []
    for node_ind in range(len(path)-1):
        from_node=path[node_ind]
        to_node=path[node_ind+1]
        link_attributes=nodes_to_link_attributes['{}_{}'.format(from_node, to_node)]
        distances+=[link_attributes['distance']]
        coords+=[link_attributes['from_coord']]
    coords+= [link_attributes['to_coord']]
    # add the final coordinate of the very last segment
    return coords, distances

# Test The Results

In [54]:
from_node, to_node=drive_edges.iloc[0]['aNodes'], drive_edges.iloc[10]['aNodes']

In [55]:
# using dijkstras
print('Shortest path by num links: {}'.format(nx.shortest_path(G, from_node, to_node)))
print('Shortest path by distance: {}'.format(nx.dijkstra_path(G, from_node, to_node, weight='weight')))
print('Distance of Shortest path: {}'.format(nx.dijkstra_path_length(G, from_node, to_node, weight='weight')))

Shortest path by num links: ['62554817', '62554820', '62554824', '5893823637', '62886081', '4516283939', '62886083', '4516283942', '62887341', '6991005588', '6991466150', '62887339', '62887336', '3324296784', '62887334', '3405663202', '6011190802', '4956715002', '4956715001', '62760467', '6011190803', '62597333', '62923577', '62893325', '4956715003', '4216058503', '5337887343', '4484794161', '62913639', '62913588', '4484794160', '5337879961', '6551679745', '6551679742', '4956715004', '63015043', '3752254074', '62688689', '4956715006', '4608468600', '62589101', '62589093', '4608468599', '6011190804', '6011190805', '62832584', '62992694', '5917153169', '62992692', '2760041224', '5917153168', '3864350194']
Shortest path by distance: ['62554817', '62554820', '62554824', '5893823637', '62886081', '5893834489', '5893834488', '62997241', '5893824534', '6991005589', '6991005590', '6991005592', '6991005591', '6991005588', '6991466150', '62887339', '62887336', '3324296784', '62887334', '34056632

In [56]:
path=get_node_path_from_fw(fw_result[0], from_node, to_node)
coords, distances=get_path_coords_distances(drive_edges, path)
print(path)
print(coords)
print(sum(distances))

['62554817', '62554820', '62554824', '5893823637', '62886081', '5893834489', '5893834488', '62997241', '5893824534', '6991005589', '6991005590', '6991005592', '6991005591', '6991005588', '6991466150', '62887339', '62887336', '3324296784', '62887334', '3405663202', '6011190802', '4956715002', '4956715001', '62760467', '6011190803', '62597333', '62923577', '62893325', '4956715003', '4216058503', '5337887343', '4484794161', '62913639', '62913588', '4484794160', '5337879961', '6551679745', '6551679742', '4956715004', '63015043', '3752254074', '62688689', '4956715006', '4608468600', '62589101', '852362503', '4691680015', '62589102', '62589104', '833652505', '4691679930', '833652964', '5589589275', '63044727', '63044730', '5589589277', '62628358', '5698337944', '5917197329', '3864350194']
[[-83.065611, 42.334337], [-83.065464, 42.334282], [-83.065374, 42.334236], [-83.065443, 42.3342145], [-83.0667672, 42.3337382], [-83.0663945, 42.3330845], [-83.0663668, 42.3330384], [-83.0663338, 42.332977

In [61]:
import datetime
count=0
then=datetime.datetime.now()
for from_node in fw_result[0]:
    print(len(fw_result[0][from_node]))
    for to_node in fw_result[0][from_node]:
        path=get_node_path_from_fw(fw_result[0], from_node, to_node)
        coords, distances=get_path_coords_distances(drive_edges, path)
        count+=1
now=datetime.datetime.now()
interval=(now-then).seconds
print('{} seconds per query'.format(interval/count))

1723
1722
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1722
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721


1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1721
1723
1722
1721
1721
1721
1721
1721
1721
1721
4.794247443264977e-05 seconds per query


In [ ]:
# Create a coordinate lookup for all the nodes included in the final graph

In [71]:
all_nodes=set(list(drive_edges['aNodes'])+list(drive_edges['bNodes']))
node_to_coordinates={}
for node in LatLongDict:
    if node in all_nodes:
        node_to_coordinates[node]=[float(LatLongDict[node][1]), float(LatLongDict[node][0])]

In [66]:
json.dump(fw_result[0], open(FLOYD_PREDECESSOR_PATH, 'w'))
drive_edges.to_csv(INT_NET_DF_FLOYD_PATH)
json.dump(node_to_coordinates, open(INT_NET_COORDINATES_PATH, 'w'))